# Reconhecimentos de Padrões
## Dataset: sEMG Basic Hand movements

### Importações

In [2]:
import scipy.io
import numpy as np
import os
from librosa import stft
from numpy import mean, sqrt, square, arange


#### Carregando o Dataset

In [21]:
caminhos = [os.path.join("src", nome) for nome in os.listdir("src")]
pessoas = []

for essive in caminhos:
    if essive != "/novo.txt":
        mat = scipy.io.loadmat(essive)
        tip = [mat['tip_ch1'], mat['tip_ch2']]
        spher = [mat['spher_ch1'], mat['spher_ch2']]
        palmar = [mat['palm_ch1'], mat['palm_ch2']]
        lateral = [mat['lat_ch1'], mat['lat_ch2']]
        cilindrical = [mat['cyl_ch1'], mat['cyl_ch2']]
        hook = [mat['hook_ch1'], mat['hook_ch2']]
        data = [tip, spher, palmar, lateral, cilindrical, hook]
        data = np.array(data)
        pessoas.append(data)  
        


dataone = pessoas[2]
dataone = np.array(dataone)
dataone = np.swapaxes(dataone,1,2)

#print(" 6 movimentos,30 tentativas, 2 canais,  3000 coletas")


#### Shape após carregamento

> 6 movimentos, 30 tentativas, 2 canais,  3000 coletas

In [22]:
print(dataone.shape)

(6, 30, 2, 3000)


#### Segmentação

In [23]:
datax = []
segmentosize = (len(dataone[0,0,0,:]))/6
salto = segmentosize * 0.7
antpasso = segmentosize - salto


print("Sobreposicao:        ", antpasso)
print("Salto:               ",salto)
print("Tamanho do segmento: ",segmentosize)


for movimento in dataone:  
    tentativs = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            
            listinha = []
            listinha.append(canal[350:850])
            listinha.append(canal[700:1200])
            listinha.append(canal[1050:1550])
            listinha.append(canal[1400:1900])
            listinha.append(canal[1750:2250])
            listinha.append(canal[2100:2600])
            canals.append(np.split(canal,5) + listinha)
        tentativs.append(canals)
    datax.append( tentativs)
datax = np.array(datax)


Sobreposicao:         150.0
Salto:                350.0
Tamanho do segmento:  500.0


#### Shape após segmentação

##### 6 movimentos, 30 tentativas, 2 canais,  11 segmentos de 500 valores

In [24]:
print(datax.shape)

(6, 30, 2, 11)


## Domínio do tempo

### MAV

In [25]:
iemg = []
for movimento in datax:
    listaar = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            for segmento in canal:
                f = lambda x: np.sum(np.abs(x)) / 500
                au = map(f, segmento)
                listaar.append(au)
    iemg.append(listaar)
iemg = np.array(iemg)
print(iemg.shape)

(6, 660)


### RMS

In [26]:
rmslist = []
for movimento in datax:
    listaar = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            for segmento in canal:
                rms = sqrt(mean(square(segmento)))
                listaar.append(rms)
        #listaar.append(canals)
    rmslist.append(listaar)
rmslist = np.array(rmslist)
print(rmslist.shape)

(6, 660)


### VAR

In [27]:
VARLIST = []
for movimento in datax:
    listaar = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            for segmento in canal:
                VAR = np.var(segmento)
                listaar.append(VAR)
    VARLIST.append(listaar)
VARLIST = np.array(VARLIST)
print(VARLIST.shape)

(6, 660)


## Domínio da frequência

#### FFT

In [41]:
print(datax[:,:,:,0:11].shape)

fft = []
for lvl1 in datax:
    ffttemp = []
    for lvl2 in lvl1:
        for lvl3 in lvl2:
            for lvl4 in lvl3:
                #, hop_length=150
                temp = np.abs(stft(lvl4, n_fft=500,hop_length=350))
                temp = temp[:,0:4]
                temp = np.array(temp)
                #print(temp.shape)
                ffttemp.append(temp)
                #print(temp.sum()/len(temp))
    fft.append(ffttemp)
fft = np.array(fft)
#fft = np.swapaxes(fft,2,3)
print(fft.shape)

(6, 30, 2, 11)
(6, 660, 251, 2)


#### PSD

In [42]:
psd = np.power(abs(fft), 1)
print(psd.shape)

(6, 660, 251, 2)


#### FMD

In [43]:
fmd = np.sum(psd/2, axis=-1)
print(fmd.shape)

(6, 660, 251)
